SILVER LAYER SCRIPTS

patient data transformation script

In [0]:
from pyspark.sql.functions import col, expr

bronze_pt_df = spark.table("bronze_fhir_patient")
# Start from flattened data
silver_df = (
    bronze_pt_df
    .withColumn("name_given", expr("name[0].given[0]"))
    .withColumn("name_family", expr("name[0].family"))
    .withColumn("identifier_system", expr("identifier[0].system"))
    .withColumn("identifier_value", expr("identifier[0].value"))
    .withColumn("address_city", expr("address[0].city"))
    .withColumn("address_postalCode", expr("address[0].postalCode"))
    .withColumn("telecom_system", expr("telecom[0].system"))
    .withColumn("telecom_value", expr("telecom[0].value"))
    .withColumn("dq_flag", expr("""
        CASE 
            WHEN id IS NULL THEN 'FAIL'
            WHEN gender NOT IN ('male', 'female', 'other', 'unknown') THEN 'FAIL'
            WHEN birthDate IS NULL THEN 'FAIL'
            ELSE 'PASS'
        END
    """))
)

# Separate good vs bad
dq_pass = silver_df.filter(col("dq_flag") == "PASS")
dq_fail = silver_df.filter(col("dq_flag") == "FAIL")

# Write valid data
dq_pass.select(
    "id", "gender", "birthDate", "name_given", "name_family",
    "identifier_system", "identifier_value", "address_city",
    "address_postalCode", "telecom_system", "telecom_value",
    "_source_file", "_ingest_time"
).write.format("delta").mode("append").saveAsTable("silver_fhir_patient")

# Optionally, store invalid records separately
dq_fail.write.format("delta").mode("append").saveAsTable("dq_errors_fhir_patient")


observation data transformation script

In [0]:
from pyspark.sql.functions import col, expr

bronze_obs_df = spark.table("bronze_fhir_observation")

silver_obs_df = (
    bronze_obs_df
    .withColumn("code_system", expr("code.coding[0].system"))
    .withColumn("code_code", expr("code.coding[0].code"))
    .withColumn("code_display", expr("code.coding[0].display"))
    .withColumn("code_text", col("code.text"))
    .withColumn("subject_reference", col("subject.reference"))
    .withColumn("value_quantity_value", expr("component[0].valueQuantity.value"))
    .withColumn("value_quantity_unit", expr("component[0].valueQuantity.unit"))
    .withColumn(
        "dq_flag",
        expr("""
            CASE
                WHEN id IS NULL THEN 'FAIL'
                WHEN status IS NULL THEN 'FAIL'
                WHEN code_code IS NULL THEN 'FAIL'
                WHEN subject_reference IS NULL THEN 'FAIL'
                WHEN effectiveDateTime IS NULL THEN 'FAIL'
                ELSE 'PASS'
            END
        """)
    )
)

# Split PASS and FAIL records
dq_pass_obs_df = silver_obs_df.filter(col("dq_flag") == "PASS").drop("dq_flag")
dq_fail_obs_df = silver_obs_df.filter(col("dq_flag") == "FAIL")

# Write PASS records to Silver table
dq_pass_obs_df.write.format("delta").mode("append").saveAsTable("silver_fhir_observation")

# Write FAIL records to a separate DQ error table
dq_fail_obs_df.write.format("delta").mode("append").saveAsTable("dq_errors_fhir_observation")
